In [13]:
# 주피터 노트북 파일로 로또 데이터 머신러닝을 활용한 예측 시도

# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# CSV 파일 불러오기
file_path = r"C:\Users\highk\pypy50\32. 응용 로또 크롤링\lotto_data_1_to_current.csv"
df = pd.read_csv(file_path)

# "1등 당첨금액" 컬럼에서 '원'을 제거하고, 정수형으로 변환 (데이터 타입을 numpy의 int64로 변환하여 큰 숫자 처리)
df["1등 당첨금액"] = df["1등 당첨금액"].str.replace('원', '').str.replace(',', '').astype(np.int64)

# "1등 당첨자수" 컬럼에서 '명'을 제거하고, 정수형으로 변환
df["1등 당첨자수"] = df["1등 당첨자수"].str.replace('명', '').astype(int)

# "추첨일"을 날짜형으로 변환
df["추첨일"] = pd.to_datetime(df["추첨일"])

# 당첨 번호를 분리 (가정: 당첨 번호가 '1등 번호' 컬럼에 쉼표로 구분된 문자열 형태로 저장되어 있음)
df_numbers = df["1등 번호"].str.split(',', expand=True)
df_numbers.columns = ["번호1", "번호2", "번호3", "번호4", "번호5", "번호6"]

# 원본 데이터프레임에 분리된 번호 컬럼 추가
df = pd.concat([df, df_numbers], axis=1)

# 번호 컬럼을 정수형으로 변환
number_columns = ["번호1", "번호2", "번호3", "번호4", "번호5", "번호6"]
df[number_columns] = df[number_columns].astype(int)

# 머신러닝 모델을 위한 데이터 준비
df['총합'] = df[number_columns].sum(axis=1)  # 예시로 각 회차 번호의 합을 특징으로 사용
X = df[['회차', '총합']]
y = df[number_columns].values  # 여러 개의 번호가 target이 됨

# 데이터셋 분할 (훈련 세트와 테스트 세트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 큰 수의 법칙에 따른 번호별 확률 계산
number_counts = df[number_columns].melt(value_name="당첨번호")["당첨번호"].value_counts().sort_index()
num_least_drawn = max(1, int(len(number_counts) * 0.2))
least_drawn_numbers = number_counts.nsmallest(num_least_drawn).index.tolist()

# 예측된 번호 중 가장 적게 뽑힌 번호에 가중치를 부여해 랜덤 추출
weighted_preds = []
for i in range(5):  # 최종 예측을 5가지 결과물로 도출
    pred_numbers = y_pred[i % len(y_pred)]
    # 기존 예측 번호에서 적게 뽑힌 번호와 결합하여 40% 확률로 섞음
    new_numbers = list(set(pred_numbers) | set(least_drawn_numbers[:int(len(least_drawn_numbers) * 0.4)]))
    weighted_preds.append(sorted(new_numbers)[:6])

# 최종 예측 번호 출력
for i, pred in enumerate(weighted_preds):
    print(f"최종 예측 번호 {i + 1} (가중치 적용):", pred)


최종 예측 번호 1 (가중치 적용): [5, 9, 11, 19, 21, 22]
최종 예측 번호 2 (가중치 적용): [5, 9, 10, 14, 22, 30]
최종 예측 번호 3 (가중치 적용): [8, 9, 22, 23, 25, 27]
최종 예측 번호 4 (가중치 적용): [3, 6, 9, 15, 20, 22]
최종 예측 번호 5 (가중치 적용): [3, 9, 10, 11, 22, 32]


In [15]:
# 주피터 노트북 파일로 로또 데이터 머신러닝을 활용한 예측 시도

# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# CSV 파일 불러오기
file_path = r"C:\Users\highk\pypy50\32. 응용 로또 크롤링\lotto_data_1_to_current.csv"
df = pd.read_csv(file_path)

# "1등 당첨금액" 컬럼에서 '원'을 제거하고, 정수형으로 변환 (데이터 타입을 numpy의 int64로 변환하여 큰 숫자 처리)
df["1등 당첨금액"] = df["1등 당첨금액"].str.replace('원', '').str.replace(',', '').astype(np.int64)

# "1등 당첨자수" 컬럼에서 '명'을 제거하고, 정수형으로 변환
df["1등 당첨자수"] = df["1등 당첨자수"].str.replace('명', '').astype(int)

# "추첨일"을 날짜형으로 변환
df["추첨일"] = pd.to_datetime(df["추첨일"])

# 당첨 번호를 분리 (가정: 당첨 번호가 '1등 번호' 컬럼에 쉼표로 구분된 문자열 형태로 저장되어 있음)
df_numbers = df["1등 번호"].str.split(',', expand=True)
df_numbers.columns = ["번호1", "번호2", "번호3", "번호4", "번호5", "번호6"]

# 원본 데이터프레임에 분리된 번호 컬럼 추가
df = pd.concat([df, df_numbers], axis=1)

# 번호 컬럼을 정수형으로 변환
number_columns = ["번호1", "번호2", "번호3", "번호4", "번호5", "번호6"]
df[number_columns] = df[number_columns].astype(int)

# 번호별 당첨 횟수 계산
draw_counts = df[number_columns].melt(value_name="당첨번호")["당첨번호"].value_counts().sort_values()

# 번호별 당첨 횟수를 테이블 형태로 출력
draw_counts_df = pd.DataFrame({'번호': draw_counts.index, '당첨 횟수': draw_counts.values}).sort_values(by='당첨 횟수')
print(draw_counts_df)

# 머신러닝 모델을 위한 데이터 준비
df['총합'] = df[number_columns].sum(axis=1)  # 예시로 각 회차 번호의 합을 특징으로 사용
X = df[['회차', '총합']]
y = df[number_columns].values  # 여러 개의 번호가 target이 됨

# 데이터셋 분할 (훈련 세트와 테스트 세트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 큰 수의 법칙에 따른 번호별 확률 계산
number_counts = df[number_columns].melt(value_name="당첨번호")["당첨번호"].value_counts().sort_index()
num_least_drawn = max(1, int(len(number_counts) * 0.2))
least_drawn_numbers = number_counts.nsmallest(num_least_drawn).index.tolist()

# 예측된 번호 중 가장 적게 뽑힌 번호에 가중치를 부여해 랜덤 추출
weighted_preds = []
for i in range(5):  # 최종 예측을 5가지 결과물로 도출
    pred_numbers = y_pred[i % len(y_pred)]
    # 기존 예측 번호에서 적게 뽑힌 번호와 결합하여 40% 확률로 섞음
    new_numbers = list(set(pred_numbers) | set(least_drawn_numbers[:int(len(least_drawn_numbers) * 0.4)]))
    weighted_preds.append(sorted(new_numbers)[:6])

# 최종 예측 번호 출력
for i, pred in enumerate(weighted_preds):
    print(f"최종 예측 번호 {i + 1} (가중치 적용):", pred)


    번호  당첨 횟수
0    9    121
1   32    133
2   22    135
3   23    136
4   41    140
5   25    140
6   28    142
7   29    143
8    5    144
9    8    144
10  42    145
12  30    146
11   2    146
13  35    150
14  36    152
15  24    152
16   4    152
17   6    153
18  15    153
19  16    153
20  10    153
21  39    153
23  26    154
22  44    154
24  31    155
25  38    155
26   3    156
27   7    156
28  11    157
29  21    157
30   1    158
31  43    158
32  19    158
34  20    159
33  37    159
35  17    161
36  27    161
37  40    163
38  45    164
39  14    165
40  18    165
41  13    165
42  33    165
43  12    168
44  34    177
최종 예측 번호 1 (가중치 적용): [5, 9, 11, 19, 21, 22]
최종 예측 번호 2 (가중치 적용): [5, 9, 10, 14, 22, 30]
최종 예측 번호 3 (가중치 적용): [8, 9, 22, 23, 25, 27]
최종 예측 번호 4 (가중치 적용): [3, 6, 9, 15, 20, 22]
최종 예측 번호 5 (가중치 적용): [3, 9, 10, 11, 22, 32]


In [3]:
# 주피터 노트북 파일로 로또 데이터 머신러닝을 활용한 예측 시도

# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# CSV 파일 불러오기
file_path = r"C:\Users\highk\pypy50\32. 응용 로또 크롤링\lotto_data_1_to_current.csv"
df = pd.read_csv(file_path)

# "1등 당첨금액" 컬럼에서 '원'을 제거하고, 정수형으로 변환 (데이터 타입을 numpy의 int64로 변환하여 큰 숫자 처리)
df["1등 당첨금액"] = df["1등 당첨금액"].str.replace('원', '').str.replace(',', '').astype(np.int64)

# "1등 당첨자수" 컬럼에서 '명'을 제거하고, 정수형으로 변환
df["1등 당첨자수"] = df["1등 당첨자수"].str.replace('명', '').astype(int)

# "추첨일"을 날짜형으로 변환
df["추첨일"] = pd.to_datetime(df["추첨일"])

# 당첨 번호를 분리 (가정: 당첨 번호가 '1등 번호' 컬럼에 쉼표로 구분된 문자열 형태로 저장되어 있음)
df_numbers = df["1등 번호"].str.split(',', expand=True)
df_numbers.columns = ["번호1", "번호2", "번호3", "번호4", "번호5", "번호6"]

# 원본 데이터프레임에 분리된 번호 컬럼 추가
df = pd.concat([df, df_numbers], axis=1)

# 번호 컬럼을 정수형으로 변환
number_columns = ["번호1", "번호2", "번호3", "번호4", "번호5", "번호6"]
df[number_columns] = df[number_columns].astype(int)

# 번호별 당첨 횟수 계산
draw_counts = df[number_columns].melt(value_name="당첨번호")["당첨번호"].value_counts().sort_values()

# 번호별 당첨 횟수를 테이블 형태로 출력
draw_counts_df = pd.DataFrame({'번호': draw_counts.index, '당첨 횟수': draw_counts.values}).sort_values(by='당첨 횟수')


# 머신러닝 모델을 위한 데이터 준비
df['총합'] = df[number_columns].sum(axis=1)  # 예시로 각 회차 번호의 합을 특징으로 사용
X = df[['회차', '총합']]
y = df[number_columns].values  # 여러 개의 번호가 target이 됨

# 데이터셋 분할 (훈련 세트와 테스트 세트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 당첨 횟수가 가장 적은 하위 10개 번호 중에서 3개 선택 (중복되지 않도록 선택)
least_drawn_numbers_top10 = draw_counts.nsmallest(10).index.tolist()
selected_least_drawn_numbers = np.random.choice(least_drawn_numbers_top10, 3, replace=False).tolist()

# 예측된 번호 중에서 나머지 2개 선택 (중복되지 않도록 선택하고 다양한 예측을 반영)
y_pred_samples = []
for i in range(5):
    y_pred_sample = y_pred[i % len(y_pred)].tolist()
    y_pred_sample = [num for num in y_pred_sample if num not in selected_least_drawn_numbers][:2]
    y_pred_samples.append(y_pred_sample)

# 최근 10번 동안 자주 등장한 당첨번호 중에서 1개 선택
recent_10_draws = df.tail(10)
recent_numbers = recent_10_draws[number_columns].melt(value_name="당첨번호")["당첨번호"].value_counts()
most_common_recent_number = recent_numbers.idxmax()

# 최종 예측 번호 생성 (3개는 적게 뽑힌 번호, 2개는 예측된 번호, 1개는 최근 자주 등장한 번호)
final_predictions = []
for i in range(5):
    final_numbers = selected_least_drawn_numbers + y_pred_samples[i] + [most_common_recent_number]
    np.random.shuffle(final_numbers)
    final_predictions.append(sorted(final_numbers))

# 최종 예측 번호 출력 (총 5가지 결과물, 예측도가 높을 것으로 기대되는 세트부터)
for i, pred in enumerate(final_predictions):
    print(f"최종 예측 번호 {i + 1} (가중치 적용):", pred)


최종 예측 번호 1 (가중치 적용): [5, 13, 21, 32, 37, 41]
최종 예측 번호 2 (가중치 적용): [5, 14, 19, 32, 37, 41]
최종 예측 번호 3 (가중치 적용): [5, 19, 22, 32, 37, 41]
최종 예측 번호 4 (가중치 적용): [3, 5, 8, 32, 37, 41]
최종 예측 번호 5 (가중치 적용): [5, 7, 18, 32, 37, 41]


In [ ]:
최종 예측 번호 1 (가중치 적용): [5, 9, 11, 19, 21, 22]
최종 예측 번호 2 (가중치 적용): [5, 9, 10, 14, 22, 30]
최종 예측 번호 3 (가중치 적용): [8, 9, 22, 23, 25, 27]
최종 예측 번호 4 (가중치 적용): [3, 6, 9, 15, 20, 22]
최종 예측 번호 5 (가중치 적용): [3, 9, 10, 11, 22, 32]
    번호  당첨 횟수
0    9    121
1   32    133
2   22    135
3   23    136
4   41    140
5   25    140
6   28    142
7   29    143
8    5    144
9    8    144
10  42    145
12  30    146
11   2    146
13  35    150
14  36    152
15  24    152
16   4    152
17   6    153
18  15    153
19  16    153
20  10    153
21  39    153
23  26    154
22  44    154
...
최종 예측 번호 2 (가중치 적용): [5, 9, 10, 14, 22, 30]
최종 예측 번호 3 (가중치 적용): [8, 9, 22, 23, 25, 27]
최종 예측 번호 4 (가중치 적용): [3, 6, 9, 15, 20, 22]
최종 예측 번호 5 (가중치 적용): [3, 9, 10, 11, 22, 32]
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...
최종 예측 번호 1 (가중치 적용): [5, 11, 12, 22, 32, 41]
최종 예측 번호 2 (가중치 적용): [5, 10, 12, 22, 32, 41]
최종 예측 번호 3 (가중치 적용): [8, 12, 22, 23, 32, 41]
최종 예측 번호 4 (가중치 적용): [3, 6, 12, 22, 32, 41]
최종 예측 번호 5 (가중치 적용): [3, 10, 12, 22, 32, 41]
---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[28], line 49
     47 df['연도'] = df['추첨일'].dt.year
     48 df['월'] = df['추첨일'].dt.month
---> 49 mean_amount_by_year = df_non_zero.groupby('연도')["1등 당첨금액"].mean().idxmax()
     50 mean_amount_by_month = df_non_zero.groupby(['연도', '월'])["1등 당첨금액"].mean().idxmax()
     52 # 가장 많은 당첨자를 배출한 연월 계산

File c:\Users\highk\anaconda3\Lib\site-packages\pandas\core\frame.py:9183, in DataFrame.groupby(self, by, axis, level, as_index, sort, group_keys, observed, dropna)
   9180 if level is None and by is None:
   9181     raise TypeError("You have to supply one of 'by' and 'level'")
-> 9183 return DataFrameGroupBy(
   9184     obj=self,
   9185     keys=by,
   9186     axis=axis,
   9187     level=level,
   9188     as_index=as_index,
   9189     sort=sort,
   9190     group_keys=group_keys,
   9191     observed=observed,
   9192     dropna=dropna,
   9193 )

File c:\Users\highk\anaconda3\Lib\site-packages\pandas\core\groupby\groupby.py:1329, in GroupBy.__init__(self, obj, keys, axis, level, grouper, exclusions, selection, as_index, sort, group_keys, observed, dropna)
...
   1044 elif isinstance(gpr, Grouper) and gpr.key is not None:
   1045     # Add key to exclusions
   1046     exclusions.add(gpr.key)

KeyError: '연도'
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...

KeyError: '연도'

In [2]:
# 주피터 노트북 파일로 로또 데이터 머신러닝을 활용한 예측 시도

# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# CSV 파일 불러오기
file_path = r"C:\Users\highk\pypy50\32. 응용 로또 크롤링\lotto_data_1_to_current.csv"
df = pd.read_csv(file_path)

# "1등 당첨금액" 컬럼에서 '원'을 제거하고, 정수형으로 변환 (데이터 타입을 numpy의 int64로 변환하여 큰 숫자 처리)
df["1등 당첨금액"] = df["1등 당첨금액"].str.replace('원', '').str.replace(',', '').astype(np.int64)

# "1등 당첨자수" 컬럼에서 '명'을 제거하고, 정수형으로 변환
df["1등 당첨자수"] = df["1등 당첨자수"].str.replace('명', '').astype(int)

# "추첨일"을 날짜형으로 변환
df["추첨일"] = pd.to_datetime(df["추첨일"])

# 당첨 번호를 분리 (가정: 당첨 번호가 '1등 번호' 컬럼에 쉼표로 구분된 문자열 형태로 저장되어 있음)
df_numbers = df["1등 번호"].str.split(',', expand=True)
df_numbers.columns = ["번호1", "번호2", "번호3", "번호4", "번호5", "번호6"]

# 원본 데이터프레임에 분리된 번호 컬럼 추가
df = pd.concat([df, df_numbers], axis=1)

# 번호 컬럼을 정수형으로 변환
number_columns = ["번호1", "번호2", "번호3", "번호4", "번호5", "번호6"]
df[number_columns] = df[number_columns].astype(int)

# 1등 당첨금액 관련 통계 계산 (최저값에서 0 제외)
df_non_zero = df[df["1등 당첨금액"] > 0]
max_amount = df_non_zero["1등 당첨금액"].max()
min_amount = df_non_zero["1등 당첨금액"].min()
mean_amount = df_non_zero["1등 당첨금액"].mean()
median_amount = df_non_zero["1등 당첨금액"].median()
upper_25_amount = df_non_zero["1등 당첨금액"].quantile(0.75)
lower_25_amount = df_non_zero["1등 당첨금액"].quantile(0.25)

# 1등 당첨자수 관련 통계 계산
max_winners = df["1등 당첨자수"].max()
min_winners = df[df["1등 당첨자수"] > 0]["1등 당첨자수"].min()

# 평균 당첨금이 가장 높았던 연도와 달 계산
df['연도'] = df['추첨일'].dt.year
df['월'] = df['추첨일'].dt.month
df_non_zero['연도'] = df_non_zero['추첨일'].dt.year
df_non_zero['월'] = df_non_zero['추첨일'].dt.month
mean_amount_by_year = df_non_zero.groupby('연도')["1등 당첨금액"].mean().idxmax()
mean_amount_by_month = df_non_zero.groupby(['연도', '월'])["1등 당첨금액"].mean().idxmax()

# 가장 많은 당첨자를 배출한 연월 계산
most_winners_month = df.groupby(['연도', '월'])["1등 당첨자수"].sum().idxmax()

# 결과 출력 (흥미로운 배경 설명 추가)
print(f"1등 당첨금액 통계:")
print(f"최고 금액: {max_amount:,}원\n- 로또 역사상 가장 큰 당첨금이었던 이 금액은 많은 사람들의 꿈을 현실로 만들어 주었습니다.")
print(f"최저 금액: {min_amount:,}원\n- 반면, 최저 당첨금은 상대적으로 적었지만 여전히 인생을 바꿀 만큼 충분한 금액이었습니다.")
print(f"평균 금액: {mean_amount:,.0f}원\n- 평균적으로, 1등 당첨자들은 이 정도 금액을 받았으며, 이는 많은 이들에게 희망을 주는 수치입니다.")
print(f"중앙값: {median_amount:,}원\n- 중앙값은 평균보다 약간 낮은 수준으로, 대다수의 당첨금이 이 금액 근처에 위치해 있음을 보여줍니다.")
print(f"상위 25%: {upper_25_amount:,}원\n- 상위 25%의 당첨금은 상당히 높은 수준이며, 큰 금액의 당첨이 종종 발생했음을 알 수 있습니다.")
print(f"하위 25%: {lower_25_amount:,}원\n- 하위 25%의 당첨금은 평균보다 낮지만 여전히 큰 금액입니다.")

print(f"\n1등 당첨자수 통계:")
print(f"최대 당첨자수: {max_winners}명\n- 가장 많은 당첨자가 나왔던 회차는 정말 특별한 날이었을 것입니다. 많은 사람들이 함께 행운을 나눈 날이었죠.")
print(f"최소 당첨자수: {min_winners}명\n- 최소 당첨자는 단 한 명이었으며, 그 한 사람이 모든 당첨금을 독차지한 놀라운 순간이었습니다.")

print(f"\n평균 당첨금이 가장 높았던 연도: {mean_amount_by_year}년\n- 이 해는 특히 큰 당첨금이 많았던 해로, 로또에 대한 관심이 더욱 뜨거웠던 시기였습니다.")
print(f"평균 당첨금이 가장 높았던 연월: {mean_amount_by_month[0]}년 {mean_amount_by_month[1]}월\n- 이 달은 특히 많은 사람들이 높은 당첨금을 기대하며 참여했던 달입니다.")
print(f"가장 많은 당첨자를 배출한 연월: {most_winners_month[0]}년 {most_winners_month[1]}월\n- 이 기간 동안은 여러 명이 함께 행운을 잡은 특별한 순간이었습니다.")

# 번호별 당첨 횟수 계산
draw_counts = df[number_columns].melt(value_name="당첨번호")["당첨번호"].value_counts().sort_values()

# 머신러닝 모델을 위한 데이터 준비
df['총합'] = df[number_columns].sum(axis=1)  # 예시로 각 회차 번호의 합을 특징으로 사용
X = df[['회차', '총합']]
y = df[number_columns].values  # 여러 개의 번호가 target이 됨

# 데이터셋 분할 (훈련 세트와 테스트 세트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 당첨 횟수가 가장 적은 하위 10개 번호 중에서 3개 선택 (중복되지 않도록 선택)
least_drawn_numbers_top10 = draw_counts.nsmallest(10).index.tolist()
selected_least_drawn_numbers = np.random.choice(least_drawn_numbers_top10, 3, replace=False).tolist()

# 예측된 번호 중에서 나머지 2개 선택 (중복되지 않도록 선택하고 다양한 예측을 반영)
y_pred_samples = []
for i in range(5):
    y_pred_sample = y_pred[i % len(y_pred)].tolist()
    y_pred_sample = [num for num in y_pred_sample if num not in selected_least_drawn_numbers][:2]
    y_pred_samples.append(y_pred_sample)

# 최근 10번 동안 자주 등장한 당첨번호 중에서 1개 선택
recent_10_draws = df.tail(10)
recent_numbers = recent_10_draws[number_columns].melt(value_name="당첨번호")["당첨번호"].value_counts()
most_common_recent_number = recent_numbers.idxmax()

# 최종 예측 번호 생성 (3개는 적게 뽑힌 번호, 2개는 예측된 번호, 1개는 최근 자주 등장한 번호)
final_predictions = []
for i in range(5):
    final_numbers = selected_least_drawn_numbers + y_pred_samples[i] + [most_common_recent_number]
    np.random.shuffle(final_numbers)
    final_predictions.append(sorted(final_numbers))

# 최종 예측 번호 출력 (총 5가지 결과물, 예측도가 높을 것으로 기대되는 세트부터)
for i, pred in enumerate(final_predictions):
    print(f"최종 예측 번호 {i + 1} (가중치 적용):", pred)


C:\Users\highk\AppData\Local\Temp\ipykernel_12792\1126289939.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_zero['연도'] = df_non_zero['추첨일'].dt.year
C:\Users\highk\AppData\Local\Temp\ipykernel_12792\1126289939.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_zero['월'] = df_non_zero['추첨일'].dt.month


1등 당첨금액 통계:
최고 금액: 40,722,959,400원
- 로또 역사상 가장 큰 당첨금이었던 이 금액은 많은 사람들의 꿈을 현실로 만들어 주었습니다.
최저 금액: 405,939,950원
- 반면, 최저 당첨금은 상대적으로 적었지만 여전히 인생을 바꿀 만큼 충분한 금액이었습니다.
평균 금액: 2,706,875,615원
- 평균적으로, 1등 당첨자들은 이 정도 금액을 받았으며, 이는 많은 이들에게 희망을 주는 수치입니다.
중앙값: 2,107,661,437.5원
- 중앙값은 평균보다 약간 낮은 수준으로, 대다수의 당첨금이 이 금액 근처에 위치해 있음을 보여줍니다.
상위 25%: 3,035,540,112.5원
- 상위 25%의 당첨금은 상당히 높은 수준이며, 큰 금액의 당첨이 종종 발생했음을 알 수 있습니다.
하위 25%: 1,596,310,855.0원
- 하위 25%의 당첨금은 평균보다 낮지만 여전히 큰 금액입니다.

1등 당첨자수 통계:
최대 당첨자수: 63명
- 가장 많은 당첨자가 나왔던 회차는 정말 특별한 날이었을 것입니다. 많은 사람들이 함께 행운을 나눈 날이었죠.
최소 당첨자수: 1명
- 최소 당첨자는 단 한 명이었으며, 그 한 사람이 모든 당첨금을 독차지한 놀라운 순간이었습니다.

평균 당첨금이 가장 높았던 연도: 2003년
- 이 해는 특히 큰 당첨금이 많았던 해로, 로또에 대한 관심이 더욱 뜨거웠던 시기였습니다.
평균 당첨금이 가장 높았던 연월: 2003년 4월
- 이 달은 특히 많은 사람들이 높은 당첨금을 기대하며 참여했던 달입니다.
가장 많은 당첨자를 배출한 연월: 2024년 7월
- 이 기간 동안은 여러 명이 함께 행운을 잡은 특별한 순간이었습니다.
최종 예측 번호 1 (가중치 적용): [9, 13, 21, 25, 37, 41]
최종 예측 번호 2 (가중치 적용): [9, 14, 19, 25, 37, 41]
최종 예측 번호 3 (가중치 적용): [9, 19, 22, 25, 37, 41]
최종 예측 번호 4 (가중치 적용): [3, 8, 9